In [2]:
!pip install qiskit
# !pip install qiskit-aer


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for qiskit from https://files.pythonhosted.org/packages/25/0c/4952744fb8a6362029afa3dda15603ab4bbd9ac57d98ea4c0c75bdb88bc2/qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata
  Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl (4.6 MB)


In [1]:
!pip install qiskit-terra qiskit-nature qiskit-algorithms
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper

# Define the molecule (simple hydrogen atom model)
molecule = Molecule(geometry=[["H", [0.0, 0.0, 0.0]]], charge=0, multiplicity=1)
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver)

# Apply transformer to freeze core orbitals
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

# Convert the electronic structure problem to a qubit Hamiltonian
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = problem.second_q_ops()[0]
qubit_hamiltonian = qubit_converter.convert(qubit_op)

# Use AerSimulator as the backend
simulator = AerSimulator()

# Define the variational form (ansatz) and optimizer
ansatz = RealAmplitudes(reps=2)
optimizer = SLSQP(maxiter=100)

# Run VQE to find the ground state energy
vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=simulator)
ground_state_result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)
ground_state_energy = ground_state_result.eigenvalue.real
print("Ground State Energy:", ground_state_energy)

# Calculate excited states by modifying the ansatz parameters
excited_energies = []
for i in range(1, 4):  # Finding 3 excited states for simplicity
    ansatz = RealAmplitudes(reps=i+1)  # Change reps to vary ansatz depth
    vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=simulator)
    result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)
    excited_energies.append(result.eigenvalue.real)
    print(f"Excited State {i} Energy:", result.eigenvalue.real)

# Calculate transition energies and intensities (proportional to 1/energy gap)
transitions = [abs(energy - ground_state_energy) for energy in excited_energies]
intensities = [1 / (energy_gap + 1e-10) for energy_gap in transitions]  # Simplified intensity

# Plot emission spectrum
plt.figure(figsize=(10, 6))
plt.bar(transitions, intensities, color='purple', width=0.02, alpha=0.7)
plt.xlabel('Transition Energy (Hartree)')
plt.ylabel('Emission Intensity (Arbitrary Units)')
plt.title('Simulated Emission Spectrum for Hydrogen Atom')
plt.show()

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for qiskit-terra from https://files.pythonhosted.org/packages/98/31/bb89a69575820815d0256055bb1abcd4d67d537c36305b6cbdca60a7145e/qiskit_terra-0.46.3-cp38-abi3-win_amd64.whl.metadata
  Using cached qiskit_terra-0.46.3-cp38-abi3-win_amd64.whl.metadata (13 kB)
  Obtaining dependency information for qiskit-nature from https://files.pythonhosted.org/packages/66/16/2bf5c270457dbda2123604108f007f4b7087fa5a3913e60f5a4f75b76d92/qiskit_nature-0.7.2-py3-none-any.whl.metadata
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
Using cached qiskit_terra-0.46.3-cp38-abi3-win_amd64.whl (5.2 MB)
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper

# Define molecule parameters (e.g., Hydrogen atom for simplicity)
molecule = Molecule(geometry=[["H", [0.0, 0.0, 0.0]]], charge=0, multiplicity=1)
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver)

# Freeze core orbitals and obtain the qubit Hamiltonian
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = problem.second_q_ops()[0]
qubit_hamiltonian = qubit_converter.convert(qubit_op)

# Set up quantum instance and VQE parameters
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
ansatz = RealAmplitudes(reps=2)
optimizer = SLSQP(maxiter=100)
vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

# Ground State Calculation
ground_state_result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)
ground_state_energy = ground_state_result.eigenvalue.real

# Excited State Calculation (Simplified: by modifying ansatz)
# For accurate excited states, a more advanced technique like eVQE would be applied.
excited_energies = []
for i in range(1, 4):  # Calculate up to 3 excited states
    ansatz = RealAmplitudes(reps=2, entanglement='linear')
    vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=quantum_instance)
    result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)
    excited_energies.append(result.eigenvalue.real)

# Calculate transition energies (energy differences)
transitions = [abs(energy - ground_state_energy) for energy in excited_energies]
intensities = [1 / (energy_diff + 1e-10) for energy_diff in transitions]  # Simplified intensity

# Plot emission spectrum
plt.figure(figsize=(10, 6))
plt.bar(transitions, intensities, color='purple', width=0.02, alpha=0.7)
plt.xlabel('Transition Energy (Hartree)')
plt.ylabel('Emission Intensity (Arbitrary Units)')
plt.title('Simulated Emission Spectrum')
plt.show()


ImportError: cannot import name 'Aer' from 'qiskit' (C:\Users\Ashwi\AppData\Roaming\Python\Python311\site-packages\qiskit\__init__.py)